# Takes data from the Capacity Market register extracted by Aurora Energy Research.


* CM register "existing" don't generally have a date attached
* Fuels inferred from info available but this involves assumptions
* Assumes that all contracted sites will get built
* This may be doubtful given the CM ruling
* https://www.carbonbrief.org/qa-what-next-for-uk-capacity-market-after-surprise-eu-ruling

In [86]:
import pandas as pd
import numpy as np
import pprint as pprint
import matplotlib.pyplot as plt
import pyproj
import re
import os

os.chdir('/Users/simon/Desktop/uk-map/')
%matplotlib inline


In [87]:
# load data extracted from the CM register by Aurora
# NB most sites have postcodes, which are not particularly accurate
# I have added lat/long columns to recips but only filled in a couple
# some sites lack location data but do mention some location info in description



# reciprocating gas/diesel engines
recip = pd.read_excel('Battery & Recip CM projects 230418.xlsx', sheetname = 'Reciprocating engines')
recip

# name the columns
recip.columns = ['blah','Site','Capacity (MW)','Postcode','Earliest CM entry','Location & description','Latitude','Longitude']

# drop the first one, which is blank
recip.drop('blah', axis=1, inplace=True)

# drop rows where Capacity is not a number or has no value
recip = recip[pd.to_numeric(recip['Capacity (MW)'], errors='coerce').notnull()]
recip.dropna(subset=['Capacity (MW)'], inplace=True)

# create columns for fuel and type
recip['Fuel'] = np.nan
recip['Type'] = 'Reciprocating engine'

# add "Gas" where the word gas appears in description
# this will mis-label some sites that mention eg "Gas works land" or "gas meter point"
recip['Fuel'] = np.where(recip['Location & description'].str.contains('Gas ')|
                         recip['Location & description'].str.contains('gas ')|
                         recip['Location & description'].str.contains('Gas-')|
                         recip['Location & description'].str.contains('gas-')|
                         recip['Location & description'].str.contains('gas.'),'Gas',recip['Fuel'])
# label diesel recips as "Oil" fuelled and type "Diesel engine"
recip['Fuel'] = np.where((recip['Location & description'].str.contains('Diesel')|
                          recip['Location & description'].str.contains('diesel')),'Oil',recip['Fuel'])
recip['Type'] = np.where((recip['Location & description'].str.contains('Diesel')|
                          recip['Location & description'].str.contains('diesel')),'Diesel engine',recip['Type'])

#recip['Location & description'].loc[recip['Location & description'].str.contains('Gas')]
#recip[['Fuel']].loc[recip['Location & description'].str.contains('Gas ')] = 'Gas'
#recip[['Fuel']].loc[recip['Location & description'].str.contains('Gas ')]
#recip

In [88]:
# import data on the lat/long of each UK postcode

post_codes = pd.read_csv('ukpostcodes.csv')

# drop lines that have lat 99.999 and long 0.000

post_codes = post_codes[post_codes.latitude!=99.999999]

In [89]:
# merge the datasets

recip2 = pd.merge(recip,post_codes,left_on=['Postcode'],right_on=['postcode'], how='left')

recip2['Latitude'] = np.where(recip2.Latitude.isnull(),recip2.latitude,recip2.Latitude)
recip2['Longitude'] = np.where(recip2.Longitude.isnull(),recip2.longitude,recip2.Longitude)

In [90]:
# create short post

recip2['short_post'] = recip2['Postcode'].str[:-1]
post_codes['short_post'] = post_codes['postcode'].str[:-1]

#find the average lat/long for each short post

avg_code = post_codes[['short_post', 'latitude', 'longitude']].groupby(['short_post']).mean().reset_index()

avg_code.columns = ['short_post','lat','lon']

In [91]:
# merge data again

recip3 = pd.merge(recip2,avg_code,left_on=['short_post'],right_on=['short_post'], how='left')

In [92]:
# use short_post avg lat/lon where no other option

recip3['Latitude'] = np.where(recip3.Latitude.isnull(),recip3.lat,recip3.Latitude)
recip3['Longitude'] = np.where(recip3.Longitude.isnull(),recip3.lon,recip3.Longitude)

In [93]:
# drop sites without a location

recip3.dropna(subset=['Latitude'],inplace=True)

recip3['Year Open'] = np.nan
recip3['Year Open'] = np.where(recip3['Earliest CM entry']>2018,2019,recip3['Year Open'])
recip3['Year Open'] = np.where(recip3['Earliest CM entry']<2018,2008,recip3['Year Open'])
recip3['Year Open'] = np.where(recip3['Earliest CM entry']==2018,2018,recip3['Year Open'])

# tidy columns

recip3.drop(['Postcode','Location & description','postcode','id','latitude','longitude','short_post','lat','lon','Earliest CM entry'], axis=1,inplace=True)


# assume fuel is gas if not stated

recip3['Fuel'].replace('nan','Gas',inplace=True)

# add start/end etc

recip3['yearStart'] = recip3['Year Open']
recip3['yearEnd'] = 2019
recip3['Renewable'] = 'No'
recip3['Low-carbon'] = 'No'


In [94]:
recip3.to_csv('recip.csv',index=False, encoding='utf-8')

# analysis and checks

In [69]:
recip3.tail()

,Site,Capacity (MW),Latitude,Longitude,Fuel,Type,Year Open,yearStart,yearEnd,Renewable,Low-carbon
303,Sidings Road,2.767,53.1084,-1.24333,Gas,Reciprocating engine,2008.0,2008.0,2019,No,No
304,A41707,2.411,53.2426,-1.33532,Gas,Reciprocating engine,2019.0,2019.0,2019,No,No
305,A41709,2.386,53.2088,-1.18345,Gas,Reciprocating engine,2019.0,2019.0,2019,No,No
306,AUKN33,2,53.6185,-1.15791,Gas,Reciprocating engine,2018.0,2018.0,2019,No,No
307,FIRST RENEWABLE ALPHA LIMITED,2,52.3994,-1.14028,Oil,Diesel engine,2019.0,2019.0,2019,No,No


In [5]:
print recip['Capacity (MW)'].loc[recip['Earliest CM entry']=='Existing'].sum()
print recip['Capacity (MW)'].loc[recip['Earliest CM entry']!='Existing'].sum()

#recip['Earliest CM entry'].unique()

464.882
3591.277
